# AI Engineer Technical Assessment

## Overview
Build an AI-powered solution for sentiment analysis of movie reviews that leverages the existing dataset to improve accuracy. This assessment is designed to be completed in 2-3 hours, we do NOT expect very detailed answers or long explanations.

## Notes
- AI assistance is allowed and, in fact, encouraged. caveats are:
    - Concise explanations and simple code are preferred
    - Solutions that use newer information and go beyond LLMs cuttof date are valuable.
    - You must be able to explain the code you write here

- Look up any information you need, copy and paste code is allowed.
- Setup the environment as needed. You can use your local environment, colab, or any other environment of your preferenc.
- Focus on working solutions, leave iteration and improvements if you have extra time.

## Setup
The following cells will download and prepare the IMDB dataset. 

In [40]:
import pandas as pd
from datasets import load_dataset

# Load IMDB dataset
dataset = load_dataset("imdb")
train_df = pd.DataFrame(dataset['train'])
test_df = pd.DataFrame(dataset['test'])

# Sample subset for quicker development
train_df = train_df.sample(n=5000, random_state=42)
test_df = test_df.sample(n=10, random_state=42)

print(f"Training samples: {len(train_df)}")
print(f"Test samples: {len(test_df)}")

# Display sample data
print("\nSample review:")
sample = train_df.iloc[0]
print(f"Text: {sample['text'][:200]}...")
print(f"Sentiment: {'Positive' if sample['label'] == 1 else 'Negative'}")

Training samples: 5000
Test samples: 10

Sample review:
Text: Dumb is as dumb does, in this thoroughly uninteresting, supposed black comedy. Essentially what starts out as Chris Klein trying to maintain a low profile, eventually morphs into an uninspired version...
Sentiment: Negative


In [41]:
train_df.head()

,text,label
6868,"Dumb is as dumb does, in this thoroughly unint...",0
24016,I dug out from my garage some old musicals and...,1
9668,After watching this movie I was honestly disap...,0
13640,This movie was nominated for best picture but ...,1
14018,Just like Al Gore shook us up with his painful...,1


## Task 1: Model Implementation
Implement a solution that analyzes sentiment in movie reviews. This part is explicitly open-ended: Explore ways to leverage the example dataset to enhance predictions. You can consider a pre-trained language model that can understand and generate text, external API's, RAG systems etc. 
Feel free to use any library or tool you are comfortable with.

To address this sentiment analysis assessment, I have chosen to evaluate two approaches:

1. **Lightweight Encoder Model (ModernBERT):** Using an encoder-only transformer model trained for sentiment classification. Specifically, I have decided to use [ModernBERT](https://arxiv.org/pdf/2412.13663), a next-generation encoder model (2024) that introduces several architectural advancements over the original BERT.
2. **Sentiment Classification with a Small LLM:** The second approach explores the use of a small-scale Large Language Model (LLM), ranging between 1B to 3B parameters, to perform sentiment classification through prompt engineering.

In [88]:
# Lightweight Encoder Model (ModernBERT)
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

class LightweightModelService:
    def __init__(self, model_name: str ="clapAI/modernBERT-base-multilingual-sentiment") -> None:
        self.device = "cuda" if torch.cuda.is_available() else "cpu"

        # Load the tokenizer and model
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name, torch_dtype=torch.float16).to(self.device).eval()

        print(f"Retrieve labels from the model's configuration  {self.model.config.id2label}")

    def predict(self, movie_review: str) -> tuple[str, float]:
        inputs = self.tokenizer(movie_review, return_tensors="pt").to(self.device)

        with torch.inference_mode():
            outputs = self.model(**inputs)
            probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
            confidence, prediction = torch.max(probs, dim=-1)

        label_map = self.model.config.id2label
        sentiment = label_map[prediction.item()].lower()  

        # Map neutral to negative for binary compatibility
        if sentiment == "neutral":
            sentiment = "negative"

        return sentiment, round(confidence.item(), 4)

In [83]:
import re
from typing import Any, cast

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline  # type: ignore

### Instructions:
# System-level prompt defining the assistant's behavior
def get_system_prompt() -> str:
    return """You are an expert in sentiment analysis of movie reviews. Your task is to evaluate the sentiment of the given review and classify it as "positive" or "negative", and provide a confidence between 0-1. Your analysis must consider both explicit and implicit sentiment cues while maintaining a focus on the target entity. 
    
    1. **Sentiment Classification Criteria**:  
    - **Positive**: The review expresses favorable opinions about the movie, highlighting its strengths or praising specific aspects.  
    - **Negative**: The review conveys unfavorable opinions, criticizing elements of the movie or expressing disappointment.

    3. **Handling Mixed Sentiments and Implicit Sentiment**:  
    - If both positive and negative elements exist, classify based on the **dominant sentiment**, considering intensity and frequency.  
    - Detect subtle tones, including **irony, implied sentiment, and framing biases** (e.g., selective comparisons, loaded phrases).  

    5. **Output Format**:  
    - Return:
        classification: <positive|negative>  
        confidence: <float between 0 and 1>
    - Do not include explanations, additional text, or punctuation.
    """

# User-level prompt defining the interaction model
def get_user_prompt(movie_review: str) -> str:
    return f"""Analyze the sentiment of the provided movie review: {movie_review}"""


class Phi4LLM:
    """
    A service class for interacting with the PHI 4 model.
    """

    def __init__(
        self, model_name: str = "microsoft/Phi-4-mini-instruct", temperature: float = 0.1, max_new_tokens: int = 500
    ) -> None:
        torch.random.manual_seed(0)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.temperature = temperature
        self.max_new_tokens = max_new_tokens

        self.model_name = model_name

        self.initialize_model()

    def initialize_model(self) -> None:

        self.model = AutoModelForCausalLM.from_pretrained(
            self.model_name,
            **self.get_model_kwargs(),
        ).eval()

        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)

        with torch.no_grad():
            self.completion_pipeline = pipeline(
                "text-generation",
                model=self.model,
                tokenizer=self.tokenizer,
            )

    def get_completions(self, movie_review: str) -> tuple[str, float]:
        """
        Generate completions based on the provided messages.
        """

        messages = [
            {"role": "system", "content": get_system_prompt()},
            {
                "role": "user",
                "content": get_user_prompt(movie_review),
            },
        ]

        generation_args = {
            "max_new_tokens": self.max_new_tokens,
            "return_full_text": False,
            "temperature": self.temperature,
            "do_sample": False,
        }

        with torch.no_grad():
            output = self.completion_pipeline(messages, **generation_args)

        return self.parse_llm_output(cast(str, output[0]["generated_text"]))
    
    def parse_llm_output(self, llm_output: str) -> tuple[str, float]:
        """
        Parse the LLM response to extract sentiment and confidence.
        """
        classification_match = re.search(r"classification:\s*(positive|negative)", llm_output, re.IGNORECASE)
        confidence_match = re.search(r"confidence:\s*([0-1](?:\.\d+)?)", llm_output)

        if classification_match and confidence_match:
            classification = classification_match.group(1).lower()
            confidence = float(confidence_match.group(1))
            return classification, confidence

        raise ValueError(f"Could not parse output: {llm_output}")

    def get_model_kwargs(self) -> dict[str, Any]:
        """
        Return default kwargs for huggingface model loading.
        """
        model_kwargs = {
            "device_map": "auto",
            "torch_dtype": "auto",
            "trust_remote_code": True,
        }
        # if self.device.type == "cuda":
        #     model_kwargs["attn_implementation"] = "flash_attention_2"

        return model_kwargs


## Task 2: API Implementation
Create a simple API using FastAPI that serves your solution. The API should accept a review text and return the sentiment analysis result.

Expected format:
```python
# Request
{
    "review_text": "This movie exceeded my expectations..."
}

# Response
{
    "sentiment": "positive",
    "confidence": 0.92,
    "similar_reviews": [
        {},
        {}
    ]
}
```

In [ ]:
import uvicorn
import threading
from datetime import datetime
from fastapi import FastAPI, HTTPException

from pydantic import BaseModel
from typing import Optional, List, Dict

# Your API implementation here
class ClassificationInputRequest(BaseModel):
    review_text: str

class ClassificationResponse(BaseModel):
    sentiment: str # Should be positive or negative
    confidence: Optional[float] 
    similar_reviews: List[Dict]

class HealthCheckResponse(BaseModel):
    status: str
    version: str
    timestamp: str  

app = FastAPI()

lightweight_model = LightweightModelService()
llm_model = Phi4LLM()

@app.post("/sentiment-classification-lightweight", response_model=ClassificationResponse)
async def sentiment_predict_lightweight(request: ClassificationInputRequest):
    try:
        result = lightweight_model.predict(request.review_text)
        sentiment, confidence = result

        return ClassificationResponse(
            sentiment=sentiment,
            confidence=confidence,
            similar_reviews=[]
        )

    except Exception as e:
        raise HTTPException(detail=str(e), status_code=400)

@app.post("/sentiment-classification-llm", response_model=ClassificationResponse)
async def sentiment_predict_llm(request: ClassificationInputRequest):
    try:
        result = llm_model.get_completions(request.review_text)
        sentiment, confidence = result

        return ClassificationResponse(
            sentiment=sentiment,
            confidence=confidence,
            similar_reviews=[]
        )
    except Exception as e:
        raise HTTPException(detail=str(e), status_code=400)
    
@app.get("/health", response_model=HealthCheckResponse)
async def health_check():
    return HealthCheckResponse(
        status="healthy",
        version="1.0.0",
        timestamp=datetime.utcnow().isoformat()
    )

# Code for run a FastAPI server inside a Jupyter notebook
# def run_api():
#     uvicorn.run(app, host="0.0.0.0", port=8000, reload=False)

# thread = threading.Thread(target=run_api, daemon=True)
# thread.start()

Retrieve labels from the model's configuration  {0: 'negative', 1: 'neutral', 2: 'positive'}


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cpu


INFO:     Started server process [18800]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:63401 - "GET /health HTTP/1.1" 200 OK


C:\Users\doram\AppData\Local\Temp\ipykernel_18800\2032029779.py:62: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp=datetime.utcnow().isoformat()
c:\Users\doram\Desktop\Machine_learning\Doramas-AI-Technical-Assessment\.venv\Lib\site-packages\transformers\generation\configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


INFO:     127.0.0.1:63428 - "POST /sentiment-classification-llm HTTP/1.1" 200 OK
INFO:     127.0.0.1:63466 - "POST /sentiment-classification-lightweight HTTP/1.1" 200 OK
INFO:     127.0.0.1:63466 - "POST /sentiment-classification-lightweight HTTP/1.1" 200 OK
INFO:     127.0.0.1:63466 - "POST /sentiment-classification-lightweight HTTP/1.1" 200 OK


## Task 3: Testing and Performance
Evaluate your solution's performance on the test set. Include:
1. Accuracy metrics (precision, recall, F1-score)
2. Inference speed (average time per prediction)

Compare performance with and without using the example data to demonstrate any improvements.

In [45]:
import time
from sklearn.metrics import classification_report

# Your testing code here

## Task 4: Deployment Strategy

1. Describe your deployment strategy considering:
   - Data storage and retrieval
   - Scalability
   - Resource requirements
   - Cost considerations

2. Create a simple Dockerfile to package your solution

In [46]:
# Write your deployment strategy here as a markdown cell
deployment_strategy = """
# Deployment Strategy

## Infrastructure
...

## Scalability Approach
...

## Model & Data Storage
...

## Resource & Cost Considerations
...
"""

print(deployment_strategy)

# Write your Dockerfile content
dockerfile_content = """
# Your Dockerfile here
...
"""

print("\nDockerfile:")
print(dockerfile_content)


# Deployment Strategy

## Infrastructure
...

## Scalability Approach
...

## Model & Data Storage
...

## Resource & Cost Considerations
...


Dockerfile:

# Your Dockerfile here
...



## Evaluation Criteria
- Implementation that can process reviews and return sentiments
- Use of extra data to improve predictions
- Proper API design
- Reasonable deployment strategy

Good luck!